In [1]:
import polars as p
import math

In [4]:
df = p.DataFrame({"a": [1.0]})
df.select(p.col("a").arctan())

a
f64
0.785398


In [5]:
cis_3h_hu = p.read_csv('R_CIS_3H_RELATIVE_HUMIDITY.csv', skip_rows=3)
cis_d_maxtemp = p.read_csv('R_CIS_D_MAX_TEMP.csv', skip_rows=3)
cis_d_mintemp = p.read_csv('R_CIS_D_MIN_TEMP.csv', skip_rows=3)
cis_3h_presmsl = p.read_csv('R_CIS_3H_PRES_MSL.csv', skip_rows=3)
cis_d_wind = p.read_csv('R_CIS_D_WIND.csv')

Measurement2 = MEAN

direction -> y = cos(dir) x = sin(y)

sumx, sumy

arctac(x/y) = degree

In [6]:
# create a dictionary to map month strings to integer values
month_dict = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}
month_reverse_dict = {v: k for k, v in month_dict.items()}

# WIND

In [90]:
def get_wind(year):
    sorted_filtered_cis_d_wind = (
        cis_d_wind.select(p.col(["DAY_OF_MONTH","MONTH","Daily_Wind_Spd","Daily_Wind_Dir"]))
        .with_columns([
            p.lit(year).alias("YEAR"),
            p.col("Daily_Wind_Dir").cos().alias("cosy"),
            p.col("Daily_Wind_Dir").sin().alias("sinx")
        ])
        .sort("DAY_OF_MONTH")
        .rename({"Daily_Wind_Spd":"Mean_Wind_Speed"})
        .filter(p.col("Mean_Wind_Speed") != 0)
        .select(
            p.col("DAY_OF_MONTH"),
            p.col("MONTH"),
            p.col("YEAR"),
            p.col("Mean_Wind_Speed"),
            p.col("cosy"),
            p.col("sinx")
        )
    )

    sumcosy = sorted_filtered_cis_d_wind.groupby("MONTH").agg(p.col("cosy").sum()).sort("MONTH")
    sumsinx = sorted_filtered_cis_d_wind.groupby("MONTH").agg(p.col("sinx").sum()).sort("MONTH")
    # compute the arctan of the ratio between sumsinx and sumcosy and convert to degrees
    arctan_value = (sumsinx["sinx"] / sumcosy["cosy"]).arctan().apply(lambda x: int(round(x * 180 / math.pi,0))).rename("arctan")
    # convert arctan_value series to a dataframe and merge with either sumcosy or sumsinx to add the "MONTH" column
    arctan_df = arctan_value.to_frame()
    df_horizontal_concat = p.concat(
        [
            arctan_df,
            sumcosy,
        ],
        how="horizontal",)
    merged_cis_d_wind_ = sorted_filtered_cis_d_wind.join(df_horizontal_concat, on="MONTH", how="left")
    merged_cis_d_wind = (
        merged_cis_d_wind_.rename({"arctan": "Mean_Wind_Direction"})
        .select(p.col(["DAY_OF_MONTH","MONTH","YEAR","Mean_Wind_Speed","Mean_Wind_Direction"]))
    )
    return merged_cis_d_wind

In [91]:
sorted_filtered_cis_d_wind = (
    cis_d_wind.select(p.col(["DAY_OF_MONTH","MONTH","Daily_Wind_Spd","Daily_Wind_Dir"]))
    .with_columns([
        p.lit(2022).alias("YEAR"),
        p.col("Daily_Wind_Dir").cos().alias("cosy"),
        p.col("Daily_Wind_Dir").sin().alias("sinx")
    ])
    .sort("DAY_OF_MONTH")
    .rename({"Daily_Wind_Spd":"Mean_Wind_Speed"})
    .filter(p.col("Mean_Wind_Speed") != 0)
    .select(
        p.col("DAY_OF_MONTH"),
        p.col("MONTH"),
        p.col("YEAR"),
        p.col("Mean_Wind_Speed"),
        p.col("cosy"),
        p.col("sinx")
    )
)

sumcosy = sorted_filtered_cis_d_wind.groupby("MONTH").agg(p.col("cosy").sum()).sort("MONTH")
sumsinx = sorted_filtered_cis_d_wind.groupby("MONTH").agg(p.col("sinx").sum()).sort("MONTH")
# compute the arctan of the ratio between sumsinx and sumcosy and convert to degrees
arctan_value = (sumsinx["sinx"] / sumcosy["cosy"]).arctan().apply(lambda x: int(round((x * 180 / math.pi) + (360 if x < 0 else 0), 0))).rename("arctan")

# convert arctan_value series to a dataframe and merge with either sumcosy or sumsinx to add the "MONTH" column
arctan_df = arctan_value.to_frame()
df_horizontal_concat = p.concat(
    [
        arctan_df,
        sumcosy,
    ],
    how="horizontal",)
merged_cis_d_wind_ = sorted_filtered_cis_d_wind.join(df_horizontal_concat, on="MONTH", how="left")
merged_cis_d_wind = (
    merged_cis_d_wind_.rename({"arctan": "Mean_Wind_Direction"})
    .select(p.col(["DAY_OF_MONTH","MONTH","YEAR","Mean_Wind_Speed","Mean_Wind_Direction"]))
)

In [47]:
test_DF = (
    merged_cis_d_wind_.rename({"arctan": "Mean_Wind_Direction"})
        .groupby(["DAY_OF_MONTH", "MONTH", "YEAR"])
        .agg([
            p.col("Mean_Wind_Direction").list(),
            p.col("Mean_Wind_Speed").first().alias("Mean_Wind_Speed")
        ])
)

In [ ]:
test_DF = (
    merged_cis_d_wind.arr()
)

In [92]:
test_DF = (
    merged_cis_d_wind_.rename({"arctan": "Mean_Wind_Direction"})
        .groupby(["DAY_OF_MONTH", "MONTH", "YEAR","Mean_Wind_Speed"], maintain_order=True)
        .agg(
            p.col("Mean_Wind_Direction").arr.concat(p.col('Mean_Wind_Speed')).flatten()
        )
)


In [89]:
def wind_list(dfwindunique): 
    df = (
        dfwindunique
            .groupby(["DAY_OF_MONTH", "MONTH", "YEAR","Mean_Wind_Speed"], maintain_order=True)
            .agg(
                p.col("Mean_Wind_Direction").arr.concat(p.col('Mean_Wind_Speed')).flatten()
            )
    )
    return df

In [93]:
test_DF

DAY_OF_MONTH,MONTH,YEAR,Mean_Wind_Speed,Mean_Wind_Direction
i64,str,i32,i64,list[i64]
1,"""Jan""",2022,11,"[67, 11]"
1,"""Feb""",2022,10,"[340, 10]"
1,"""Mar""",2022,9,"[298, 9]"
1,"""Apr""",2022,12,"[86, 12]"
1,"""May""",2022,10,"[309, 10]"
1,"""Jun""",2022,9,"[332, 9]"
1,"""Jul""",2022,12,"[319, 12]"
1,"""Aug""",2022,7,"[79, 7]"
1,"""Sep""",2022,8,"[305, 8]"


In [87]:
test_DF.write_json("path.json")

In [43]:
test_DF = test_DF.select([
    "DAY_OF_MONTH",
    "MONTH",
    "YEAR",
    p.col("list").alias("Mean_Wind_Direction"),
    "Mean_Wind_Speed"
])

ColumnNotFoundError: list

Error originated just after this operation:
DF ["DAY_OF_MONTH", "MONTH", "YEAR", "Mean_Wind_Direction"]; PROJECT */5 COLUMNS; SELECTION: "None"

In [21]:
merged_cis_d_wind

DAY_OF_MONTH,MONTH,YEAR,Mean_Wind_Speed,Mean_Wind_Direction
i64,str,i32,f64,i64
1,"""Jan""",2022,8.3,67
1,"""Feb""",2022,8.9,340
1,"""Mar""",2022,9.0,298
1,"""Apr""",2022,9.1,86
1,"""May""",2022,10.1,309
1,"""Jun""",2022,10.1,332
1,"""Jul""",2022,10.3,319
1,"""Aug""",2022,9.7,79
1,"""Sep""",2022,8.7,305


In [10]:
sumsinx

MONTH,sinx
str,f64
"""Apr""",3.478416
"""Aug""",-8.57642
"""Dec""",1.802061
"""Feb""",-1.411233
"""Jan""",2.536931
"""Jul""",-4.667675
"""Jun""",-3.530456
"""Mar""",4.291991
"""May""",-1.729882


In [9]:
sumcosy

MONTH,cosy
str,f64
"""Apr""",0.260349
"""Aug""",-1.611469
"""Dec""",9.753338
"""Feb""",3.888738
"""Jan""",1.080903
"""Jul""",5.307476
"""Jun""",6.578039
"""Mar""",-2.31629
"""May""",1.397407


In [10]:
merged_cis_d_wind

DAY_OF_MONTH,MONTH,YEAR,Mean_Wind_Speed,Mean_Wind_Direction
i64,str,i32,f64,i64
1,"""Jan""",2022,8.3,67
1,"""Feb""",2022,8.9,-20
1,"""Mar""",2022,9.0,-62
1,"""Apr""",2022,9.1,86
1,"""May""",2022,10.1,-51
1,"""Jun""",2022,10.1,-28
1,"""Jul""",2022,10.3,-41
1,"""Aug""",2022,9.7,79
1,"""Sep""",2022,8.7,-55


# PRESSURE

In [177]:
def get_mean_press(year,month):
    sorted_filtered_cis_3h_presmsl = (
        cis_3h_presmsl.filter(p.col('HOURS') == "22:00")
        .select(p.col(["DAY_OF_MONTH","Mean_of_Day"]))
        .with_columns([
            p.lit(month).alias("MONTH"), 
            p.lit(year).alias("YEAR")
        ])
        .sort("DAY_OF_MONTH")
        .rename({"Mean_of_Day":"Mean_Pressure_Day"})
        .select(
            p.col("DAY_OF_MONTH"),
            p.col("MONTH"),
            p.col("YEAR"),
            p.col("Mean_Pressure_Day")
        )
    )
    return sorted_filtered_cis_3h_presmsl

In [4]:
sorted_filtered_cis_3h_presmsl = (
    cis_3h_presmsl.filter(p.col('HOURS') == "22:00")
    .select(p.col(["DAY_OF_MONTH","Mean_of_Day"]))
    .with_columns([
        p.lit("Jan").alias("MONTH"), 
        p.lit(2022).alias("YEAR")
    ])
    .sort("DAY_OF_MONTH")
    .rename({"Mean_of_Day":"Mean_Pressure_Day"})
    .select(
        p.col("DAY_OF_MONTH"),
        p.col("MONTH"),
        p.col("YEAR"),
        p.col("Mean_Pressure_Day")
    )
)


In [12]:
sorted_filtered_cis_3h_presmsl

DAY_OF_MONTH,MONTH,YEAR,Mean_Pressure_Day
i64,str,i32,f64
1,"""Jan""",2022,1009.7
2,"""Jan""",2022,1009.2
3,"""Jan""",2022,1009.5
4,"""Jan""",2022,1009.9
5,"""Jan""",2022,1010.3
6,"""Jan""",2022,1010.0
7,"""Jan""",2022,1010.1
8,"""Jan""",2022,1010.1
9,"""Jan""",2022,1009.7


# HUMID

In [178]:
def get_mean_humi(year,month):
    sorted_filtered_cis_3h_hu = (
        cis_3h_hu.filter(p.col('HOURS') == "22:00")
        .select(p.col(["DAY_OF_MONTH","Mean_of_Day"]))
        .with_columns([
            p.lit(month).alias("MONTH"), 
            p.lit(year).alias("YEAR")
        ])
        .sort("DAY_OF_MONTH")
        .rename({"Mean_of_Day":"Mean_Humid_Day"})
        .select(
            p.col("DAY_OF_MONTH"),
            p.col("MONTH"),
            p.col("YEAR"),
            p.col("Mean_Humid_Day")
        )
    )
    return sorted_filtered_cis_3h_hu

In [5]:
sorted_filtered_cis_3h_hu = (
    cis_3h_hu.filter(p.col('HOURS') == "22:00")
    .select(p.col(["DAY_OF_MONTH","Mean_of_Day"]))
    .with_columns([
        p.lit("Jan").alias("MONTH"), 
        p.lit(2022).alias("YEAR")
    ])
    .sort("DAY_OF_MONTH")
    .rename({"Mean_of_Day":"Mean_Humid_Day"})
    .select(
        p.col("DAY_OF_MONTH"),
        p.col("MONTH"),
        p.col("YEAR"),
        p.col("Mean_Humid_Day")
    )
)


In [13]:
sorted_filtered_cis_3h_hu

DAY_OF_MONTH,MONTH,YEAR,Mean_Humid_Day
i64,str,i32,i64
1,"""Jan""",2022,78
2,"""Jan""",2022,73
3,"""Jan""",2022,67
4,"""Jan""",2022,69
5,"""Jan""",2022,70
6,"""Jan""",2022,68
7,"""Jan""",2022,65
8,"""Jan""",2022,70
9,"""Jan""",2022,70


# MIN

In [179]:
def get_min_temp(year):
    sorted_filtered_cis_d_mintemp = (
        cis_d_mintemp.select(p.col(["DAY_OF_MONTH","MONTH","Daily_Min_Temp"]))
        .with_columns([
            p.lit(year).alias("YEAR")
        ])
        .sort("DAY_OF_MONTH")
        .select(
            p.col("DAY_OF_MONTH"),
            p.col("MONTH"),
            p.col("YEAR"),
            p.col("Daily_Min_Temp")
        )
    )
    return sorted_filtered_cis_d_mintemp

In [6]:
sorted_filtered_cis_d_mintemp = (
    cis_d_mintemp.select(p.col(["DAY_OF_MONTH","MONTH","Daily_Min_Temp"]))
    .with_columns([
        p.lit(2022).alias("YEAR")
    ])
    .sort("DAY_OF_MONTH")
    .select(
        p.col("DAY_OF_MONTH"),
        p.col("MONTH"),
        p.col("YEAR"),
        p.col("Daily_Min_Temp")
    )
)


In [ ]:
sorted_filtered_cis_d_mintemp

# MAX

In [180]:
def get_max_temp(year):
    sorted_filtered_cis_d_maxtemp = (
        cis_d_maxtemp.select(p.col(["DAY_OF_MONTH","MONTH","Daily_Max_Temp"]))
        .with_columns([
            p.lit(year).alias("YEAR")
        ])
        .sort("DAY_OF_MONTH")
        .select(
            p.col("DAY_OF_MONTH"),
            p.col("MONTH"),
            p.col("YEAR"),
            p.col("Daily_Max_Temp")
        )
    )
    return sorted_filtered_cis_d_maxtemp

In [7]:
sorted_filtered_cis_d_maxtemp = (
    cis_d_maxtemp.select(p.col(["DAY_OF_MONTH","MONTH","Daily_Max_Temp"]))
    .with_columns([
        p.lit(2022).alias("YEAR")
    ])
    .sort("DAY_OF_MONTH")
    .select(
        p.col("DAY_OF_MONTH"),
        p.col("MONTH"),
        p.col("YEAR"),
        p.col("Daily_Max_Temp")
    )
)


In [41]:
sorted_filtered_cis_d_mintemp

DAY_OF_MONTH,MONTH,YEAR,Daily_Min_Temp
i64,str,i32,f64
1,"""Jan""",2022,22.4
1,"""Feb""",2022,26.4
1,"""Mar""",2022,26.7
1,"""Apr""",2022,27.2
1,"""May""",2022,27.2
1,"""Jun""",2022,27.2
1,"""Jul""",2022,27.6
1,"""Aug""",2022,25.5
1,"""Sep""",2022,24.7


# Merge Temp

In [181]:
def get_merge_temp(sorted_filtered_cis_d_mintemp,sorted_filtered_cis_d_maxtemp):
    merge_temp = sorted_filtered_cis_d_mintemp.join(sorted_filtered_cis_d_maxtemp, on=["DAY_OF_MONTH","MONTH"], how="left")
    merge_temp = (
        merge_temp.select(p.col(["DAY_OF_MONTH","MONTH","YEAR","Daily_Min_Temp","Daily_Max_Temp"]))
        .with_columns([
            ((p.col("Daily_Min_Temp") + p.col("Daily_Max_Temp")) / 2).alias("Daily_Mean_Temp")
        ])
    )
    return merge_temp

In [46]:
merge_temp = sorted_filtered_cis_d_mintemp.join(sorted_filtered_cis_d_maxtemp, on=["DAY_OF_MONTH","MONTH"], how="left")

In [47]:
merge_temp = (
    merge_temp.select(p.col(["DAY_OF_MONTH","MONTH","YEAR","Daily_Min_Temp","Daily_Max_Temp"]))
    .with_columns([
        ((p.col("Daily_Min_Temp") + p.col("Daily_Max_Temp")) / 2).alias("Daily_Mean_Temp")
    ])
)


# WHOLE

In [174]:
def get_whole(sorted_filtered_cis_3h_presmsl,merge_temp,merged_cis_d_wind,sorted_filtered_cis_3h_hu):
    final_data = (
        sorted_filtered_cis_3h_presmsl
        .join(merge_temp.select(p.exclude("YEAR")).with_columns(p.col("MONTH").map_dict(month_dict)).sort("MONTH").with_columns(p.col("MONTH").map_dict(month_reverse_dict)), on=["DAY_OF_MONTH","MONTH"], how="inner")
        .join(merged_cis_d_wind.select(p.exclude("YEAR")).with_columns(p.col("MONTH").map_dict(month_dict)).sort("MONTH").with_columns(p.col("MONTH").map_dict(month_reverse_dict)), on=["DAY_OF_MONTH","MONTH"], how="inner")
        .join(sorted_filtered_cis_3h_hu.select(p.exclude("YEAR")).with_columns(p.col("MONTH").map_dict(month_dict)).sort("MONTH").with_columns(p.col("MONTH").map_dict(month_reverse_dict)), on=["DAY_OF_MONTH","MONTH"], how="inner")
    )
    return final_data

In [159]:
final_data = (
    sorted_filtered_cis_3h_presmsl
    .join(merge_temp.select(p.exclude("YEAR")).with_columns(p.col("MONTH").map_dict(month_dict)).sort("MONTH").with_columns(p.col("MONTH").map_dict(month_reverse_dict)), on=["DAY_OF_MONTH","MONTH"], how="inner")
    .join(merged_cis_d_wind.select(p.exclude("YEAR")).with_columns(p.col("MONTH").map_dict(month_dict)).sort("MONTH").with_columns(p.col("MONTH").map_dict(month_reverse_dict)), on=["DAY_OF_MONTH","MONTH"], how="inner")
    .join(sorted_filtered_cis_3h_hu.select(p.exclude("YEAR")).with_columns(p.col("MONTH").map_dict(month_dict)).sort("MONTH").with_columns(p.col("MONTH").map_dict(month_reverse_dict)), on=["DAY_OF_MONTH","MONTH"], how="inner")
)

# DEMO WHOLE CALLED

In [184]:
def get_all(year,month):
    data = get_whole(get_mean_press(year,month),get_merge_temp(get_min_temp(year),get_max_temp(year)),get_wind(year),get_mean_humi(year,month))
    return data

In [185]:
year = 2022
month = "Jan"
# create a dictionary to map month strings to integer values
month_dict = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}
month_reverse_dict = {v: k for k, v in month_dict.items()}
data = get_all(year,month)

# QA Check

In [190]:
from polars.testing import assert_frame_equal

In [192]:
print(assert_frame_equal(data,final_data))

None


In [188]:
data

DAY_OF_MONTH,MONTH,YEAR,Mean_Pressure_Day,Daily_Min_Temp,Daily_Max_Temp,Daily_Mean_Temp,Mean_Wind_Speed,Mean_Wind_Direction,Mean_Humid_Day
i64,str,i32,f64,f64,f64,f64,f64,i64,i64
1,"""Jan""",2022,1009.7,22.4,30.6,26.5,8.3,67,78
2,"""Jan""",2022,1009.2,22.3,31.1,26.7,8.3,67,73
3,"""Jan""",2022,1009.5,22.4,31.7,27.05,8.3,67,67
4,"""Jan""",2022,1009.9,23.4,32.8,28.1,8.3,67,69
5,"""Jan""",2022,1010.3,22.4,33.9,28.15,8.3,67,70
6,"""Jan""",2022,1010.0,21.9,34.4,28.15,8.3,67,68
7,"""Jan""",2022,1010.1,23.2,34.4,28.8,8.3,67,65
8,"""Jan""",2022,1010.1,23.3,34.5,28.9,8.3,67,70
9,"""Jan""",2022,1009.7,22.3,34.0,28.15,8.3,67,70


In [187]:
final_data

DAY_OF_MONTH,MONTH,YEAR,Mean_Pressure_Day,Daily_Min_Temp,Daily_Max_Temp,Daily_Mean_Temp,Mean_Wind_Speed,Mean_Wind_Direction,Mean_Humid_Day
i64,str,i32,f64,f64,f64,f64,f64,i64,i64
1,"""Jan""",2022,1009.7,22.4,30.6,26.5,8.3,67,78
2,"""Jan""",2022,1009.2,22.3,31.1,26.7,8.3,67,73
3,"""Jan""",2022,1009.5,22.4,31.7,27.05,8.3,67,67
4,"""Jan""",2022,1009.9,23.4,32.8,28.1,8.3,67,69
5,"""Jan""",2022,1010.3,22.4,33.9,28.15,8.3,67,70
6,"""Jan""",2022,1010.0,21.9,34.4,28.15,8.3,67,68
7,"""Jan""",2022,1010.1,23.2,34.4,28.8,8.3,67,65
8,"""Jan""",2022,1010.1,23.3,34.5,28.9,8.3,67,70
9,"""Jan""",2022,1009.7,22.3,34.0,28.15,8.3,67,70


In [182]:
year = 2022
month = "Jan"
# create a dictionary to map month strings to integer values
month_dict = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}
month_reverse_dict = {v: k for k, v in month_dict.items()}
get_whole(get_mean_press(year,month),get_merge_temp(get_min_temp(year),get_max_temp(year)),get_wind(year),get_mean_humi(year,month))

DAY_OF_MONTH,MONTH,YEAR,Mean_Pressure_Day,Daily_Min_Temp,Daily_Max_Temp,Daily_Mean_Temp,Mean_Wind_Speed,Mean_Wind_Direction,Mean_Humid_Day
i64,str,i32,f64,f64,f64,f64,f64,i64,i64
1,"""Jan""",2022,1009.7,22.4,30.6,26.5,8.3,67,78
2,"""Jan""",2022,1009.2,22.3,31.1,26.7,8.3,67,73
3,"""Jan""",2022,1009.5,22.4,31.7,27.05,8.3,67,67
4,"""Jan""",2022,1009.9,23.4,32.8,28.1,8.3,67,69
5,"""Jan""",2022,1010.3,22.4,33.9,28.15,8.3,67,70
6,"""Jan""",2022,1010.0,21.9,34.4,28.15,8.3,67,68
7,"""Jan""",2022,1010.1,23.2,34.4,28.8,8.3,67,65
8,"""Jan""",2022,1010.1,23.3,34.5,28.9,8.3,67,70
9,"""Jan""",2022,1009.7,22.3,34.0,28.15,8.3,67,70


In [160]:
final_data

DAY_OF_MONTH,MONTH,YEAR,Mean_Pressure_Day,Daily_Min_Temp,Daily_Max_Temp,Daily_Mean_Temp,Mean_Wind_Speed,Mean_Wind_Direction,Mean_Humid_Day
i64,str,i32,f64,f64,f64,f64,f64,i64,i64
1,"""Jan""",2022,1009.7,22.4,30.6,26.5,8.3,67,78
2,"""Jan""",2022,1009.2,22.3,31.1,26.7,8.3,67,73
3,"""Jan""",2022,1009.5,22.4,31.7,27.05,8.3,67,67
4,"""Jan""",2022,1009.9,23.4,32.8,28.1,8.3,67,69
5,"""Jan""",2022,1010.3,22.4,33.9,28.15,8.3,67,70
6,"""Jan""",2022,1010.0,21.9,34.4,28.15,8.3,67,68
7,"""Jan""",2022,1010.1,23.2,34.4,28.8,8.3,67,65
8,"""Jan""",2022,1010.1,23.3,34.5,28.9,8.3,67,70
9,"""Jan""",2022,1009.7,22.3,34.0,28.15,8.3,67,70


In [156]:
sorted_filtered_cis_3h_presmsl

DAY_OF_MONTH,MONTH,YEAR,Mean_Pressure_Day
i64,str,i32,f64
1,"""Jan""",2022,1009.7
2,"""Jan""",2022,1009.2
3,"""Jan""",2022,1009.5
4,"""Jan""",2022,1009.9
5,"""Jan""",2022,1010.3
6,"""Jan""",2022,1010.0
7,"""Jan""",2022,1010.1
8,"""Jan""",2022,1010.1
9,"""Jan""",2022,1009.7


In [158]:
merge_temp.with_columns(p.col("MONTH").map_dict(month_dict)).sort("MONTH").with_columns(p.col("MONTH").map_dict(month_reverse_dict))

DAY_OF_MONTH,MONTH,YEAR,Daily_Min_Temp,Daily_Max_Temp,Daily_Mean_Temp
i64,str,i32,f64,f64,f64
1,"""Jan""",2022,22.4,30.6,26.5
2,"""Jan""",2022,22.3,31.1,26.7
3,"""Jan""",2022,22.4,31.7,27.05
4,"""Jan""",2022,23.4,32.8,28.1
5,"""Jan""",2022,22.4,33.9,28.15
6,"""Jan""",2022,21.9,34.4,28.15
7,"""Jan""",2022,23.2,34.4,28.8
8,"""Jan""",2022,23.3,34.5,28.9
9,"""Jan""",2022,22.3,34.0,28.15


In [157]:
merge_temp.with_columns(p.col("MONTH").map_dict(month_dict))

DAY_OF_MONTH,MONTH,YEAR,Daily_Min_Temp,Daily_Max_Temp,Daily_Mean_Temp
i64,i64,i32,f64,f64,f64
1,1,2022,22.4,30.6,26.5
1,2,2022,26.4,32.6,29.5
1,3,2022,26.7,34.4,30.55
1,4,2022,27.2,35.9,31.55
1,5,2022,27.2,33.5,30.35
1,6,2022,27.2,35.2,31.2
1,7,2022,27.6,33.9,30.75
1,8,2022,25.5,31.2,28.35
1,9,2022,24.7,34.3,29.5


In [95]:
final_data

DAY_OF_MONTH,MONTH,YEAR,Mean_Pressure_Day,Daily_Min_Temp,Daily_Max_Temp,Daily_Mean_Temp
i64,str,i32,f64,f64,f64,f64
1,"""Jan""",2022,1009.7,22.4,30.6,26.5
1,"""Jan""",2022,1009.7,22.3,31.1,26.7
1,"""Jan""",2022,1009.7,22.4,31.7,27.05
1,"""Jan""",2022,1009.7,23.4,32.8,28.1
1,"""Jan""",2022,1009.7,22.4,33.9,28.15
1,"""Jan""",2022,1009.7,21.9,34.4,28.15
1,"""Jan""",2022,1009.7,23.2,34.4,28.8
1,"""Jan""",2022,1009.7,23.3,34.5,28.9
1,"""Jan""",2022,1009.7,22.3,34.0,28.15
